# Representing text for Machine-learning

A long document made up of words is not a good representation for computers.

We need to convert the text into a better representation. Here we will look at
some baseline methods that work very well in practice. They also form the basis
of more complicated ideas used later on. Strong baselines are important in a world
of deep learning. You have to be able to demosntrate that the additional complexity,
reduced explainability, and additional technical debt from using deep learning is
worth it.

In [ ]:
sentences = ["The Uber driver behind the wheel of an autonomous car that hit "
             "and killed a pedestrian in Arizona could have avoided the collision "
             "if she had not been distracted, according to police investigating "
             "the incident.",
             "An avoidability analysis by police in Tempe, Arizona following March's"
             " crash suggested that Rafaela Vasquez, Uber's safety driver, may have "
             "been watching the online video service Hulu in the car.",
             "The death of 49-year-old Elaine Herzberg is believed to be the first "
             "time an autonomous car has killed a bystander, prompting a series of "
             "investigations into what happened.",
            ]

In [ ]:
# create a bag of words representation of the above sentences
# Build an `dict` that can map from a word to an integer
# use all sentences to build this dictionary
# Use it to convert each sentence into a sequence of integers
# Encode each integer in a one-hot fashion (check the keras notebook if you
# need a refresher)
# sum the individual one-hot encoded words in a sentence to produce your
# bag of words (BoW)

In [ ]:
# check that you can undo your bag of words
# create an inverse index (dictionary) that maps from
# integers back to the word.

In [ ]:
# what fraction of entries are not zero in X?

In [ ]:
(X>0).sum(axis=1) / X.shape[1]

This is a fairly small vocabulary and just three sentences but already most of the entries in the vector are zeros. Note that the (cosine) distance between two vectors for individual words carries no semantic meaning.

## Using ready made tools for this

scikit-learn has tools to do this for you.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# ... use an instance of CountVectorizer() to vectorise your sentences
vect = CountVectorizer()

print(sorted(vect.get_feature_names()))

/Users/thead/anaconda/envs/deep-nlp-class/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


NameError: name 'vect' is not defined

In [ ]:
# use the learnt mapping to convert sentences to bag of words
# CountVectorizer is a transformer. It has a `transform()`
# method that you can use after fitting it.

In [ ]:
# transform the vector representation back to "words"
# there is a helper to do this.

In [ ]:
# Is there a difference between what `CountVectorizer` says how many features (words)
# there are and your manual code?

## Classify movies from their reviews

Fetch the dataset from http://ai.stanford.edu/~amaas/data/sentiment/ and un'tar it to
a directory near to this notebook. I placed my copy in `../data/`, however it is too
large to distribute with the repository so you will have to get your own copy.

In [1]:
from sklearn.preprocessing import CountVectorizer
from sklearn.datasets import load_files

reviews_train = load_files("../data/aclImdb/train/", categories=['neg', 'pos'])

text_trainval, y_trainval = reviews_train.data, reviews_train.target

print("type of text_train: {}".format(type(text_trainval)))
print("length of text_train: {}".format(len(text_trainval)))
print("class balance: {}".format(np.bincount(y_trainval)))

ImportError: cannot import name 'CountVectorizer'

In [ ]:
print("classes: {}".format(set(y_trainval)))

In [ ]:
# look at an example review
print("text_train[42]:\n{}".format(text_trainval[42].decode()))

In [ ]:
# Vectorise the review texts!
from sklearn.model_selection import train_test_split


# sort out some HTML tags. and make a list of all training and
# validation reviews
text_trainval = [doc.replace(b"<br />", b" ").decode() for doc in text_trainval]

# Split the data into training and validation data.
# Use CountVectorizer() to fit and transform the training
# data and transform the validation data
# ... your code here

In [ ]:
# check out some of the words
feature_names = vect.get_feature_names()
print(feature_names[:10])
print(feature_names[30000:30010])
print(feature_names[::3000])

In [ ]:
from sklearn.linear_model import LogisticRegression

# fit a logistic regression model to the data and measure the performance
# of the model. We now have a baseline we need to improve on.
# What score do you achieve on which dataset? Did you split into training
# and testing datasets?
# 
# Once you have your model running tune the regularisation strength `C`
# using `RandomSearchCV` or `LogisticRegressionCV` which is more efficient
# Take a look at the scikit-learn documentation for each of the
# classes.

In [ ]:
lr.score(X_val, y_val)

In [ ]:
# Find out which words correlate with a good and a bad review.
# You can inspect the weights of the linear model by looking
# at the coefs_ property of your LogisticRegression instance
# the sign and magnitude of a coefficient tells you how much
# and in which direction the model uses each feature for
# its decisions
# Check the documentation to learn about all the properties:
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#
# do they seem sensible? Could you explain why a review is
# getting a high or a low predicted sentiment?

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14

In [ ]:
# ... your code here ...
# checkout `plt.bar()` to make a plot of the twenty most
# negative and twenty most positive words

## Bonus

* investigate how to configure the vectorizer to exclude stop words
* should you fix the spelling of misspelt words?
* only include words that appear more than N times?
* only include the M most frequent words?
* how few examples do you need to achieve a "good" performance? Study the score as a function of the number of reviews you train on
* a bag of words does not know anything about the order of words,
  can you construct bi-grams (pairs of words) and improve the
  performance? Check the documentation for `CountVectorizer`
* construct a logistic regression model in keras and use that instead
  of using scikit-learn's implementation


# Term frequency, inverse document frequency

Instead of counting how often a word appears we can also use TfIdf.

Check out the scikit-learn documentation http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


# Pipelines are extremly useful tools
# http://scikit-learn.org/stable/modules/pipeline.html
text_classifier = make_pipeline(
    TfidfVectorizer(min_df=3, max_df=0.8, ngram_range=(1, 2)),
    LogisticRegression(),
)

In [ ]:
%%time
text_classifier.fit(text_train, y_train)

In [ ]:
text_classifier.score(text_val, y_val)

## Baseline results

With about 20s of computer time and a few minutes of work we are at 88% accuracy.

We have not really tuned this baseline yet, it is possible that some investment in 
finding better hyperparamters will improve the baseline further. Try it out if you have time
by looking at `RandomizedSearchCV` http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html